Họ tên: Nguyễn Trung Dũng

MSSV: 19120486

# HW3: Các loại bộ nhớ trong CUDA

Với các GPU tương đối mới thì để biên dịch chỉ cần dùng câu lệnh: \
`nvcc tên-file.cu -o tên-file-chạy`

Nhưng trên Colab mình thường lấy được GPU khá cũ là Tesla K80 với compute capability (phiên bản phần cứng) là 3.7; để biên dịch đúng với GPU khá cũ này thì bạn cần dùng câu lệnh: \
`nvcc -arch=sm_37 tên-file.cu -o tên-file-chạy` \
Trong đó, 37 chính là compute capability của GPU Tesla K80.

Để phòng trường hợp khi làm bài bạn lấy được GPU có compute capability x.x nhưng khi chấm bài Thầy lại lấy được GPU có compute capability khác x.x, dưới đây mình sẽ có đoạn code Python để tự động lấy 2 con số ứng với compute capability của GPU và lưu vào 2 biến `major` và `minor`:


In [24]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


Một khi đã chạy đoạn code Python ở trên, để biên dịch thì bạn sẽ dùng câu lệnh: \
`nvcc -arch=sm_{major}{minor} tên-file.cu -o tên-file-chạy`

Dưới đây, khi làm bài thì bạn có thể tùy ý thêm/xóa cell. Đừng xóa mấy cell có chữ của Thầy là được.

In [25]:
!nvcc -arch=sm_{major}{minor} HW3.cu -o HW3

In [26]:
!./HW3 in.pnm out.pnm 32 32

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 32x32, grid size 16x16
Kernel time: 0.350240 ms
Error: 0.000703

Kernel 2, block size 32x32, grid size 16x16
Kernel time: 0.349216 ms
Error: 0.000703

Kernel 3, block size 32x32, grid size 16x16
Kernel time: 0.266208 ms
Error: 0.000703


In [27]:
!./HW3 in.pnm out.pnm 16 16

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 16x16, grid size 32x32
Kernel time: 0.461376 ms
Error: 0.000703

Kernel 2, block size 16x16, grid size 32x32
Kernel time: 0.319616 ms
Error: 0.000703

Kernel 3, block size 16x16, grid size 32x32
Kernel time: 0.247776 ms
Error: 0.000703


In [28]:
!./HW3 in.pnm out.pnm 16 32

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 16x32, grid size 32x16
Kernel time: 0.471072 ms
Error: 0.000703

Kernel 2, block size 16x32, grid size 32x16
Kernel time: 0.315936 ms
Error: 0.000703

Kernel 3, block size 16x32, grid size 32x16
Kernel time: 0.245920 ms
Error: 0.000703


In [29]:
!./HW3 in.pnm out.pnm 45 22

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 45x22, grid size 12x24
Kernel time: 0.427904 ms
Error: 0.000703

Kernel 2, block size 45x22, grid size 12x24
Kernel time: 0.346848 ms
Error: 0.000703

Kernel 3, block size 45x22, grid size 12x24
Kernel time: 0.272640 ms
Error: 0.000703


Có thể thấy khi dùng SMEM (kernel 2) thì hầu như là nhanh hơn so với khi không dùng SMEM (kernel 1) và khi kết hợp với CMEM (kernel 3) thì thời gian chạy cũng rút ngắn đáng kể so với kernel 1 và 2. <br/>
Trong kernel 1, ta thấy mỗi thread phải truy xuất GMEM (mảng `inPixels`) nhiều lần, làm giảm tốc độ. <br/>
Khi dùng SMEM thì ta có thể giảm số lân phải truy xuất GMEM bằng cách đọc 1 lần và lưu các phần tử phải truy xuất giá trị nhiều lần xuống bộ nhớ SMEM; hơn nữa, SMEM có tốc độ truy xuất nhanh hơn rất nhiều so với DRAM nên giảm thời gian chạy so với khi không dùng SMEM. <br/>
CMEM tăng tốc kernel theo cách tương tự như SMEM. Trong kernel 1 và 2, mảng `d_filter` được lưu ở GMEM; trong kernel 3 thì mảng `dc_filter` là thuộc CMEM, và vì có kích thước < 8KB nên chương trình có thể tận dụng bộ nhớ constant cache, có tốc độ truy xuất nhanh hơn DRAM.